# Introduction

This interactive document, along with the accompanying lecture, attempts to give a very high-level overview of how we solve partial differential equations using computers.  This is a *very* large topic, and the aspects touched upon here would cover several college (possibly graduate level) courses.  The principle goal is to give the reader a basic understanding of the major concepts, and, in particular, the differences (and similarities) between finite difference, finite volume, and finite volume methods of solution.

There are a lot of critical topics which are left out, which will be noted at the end.

# Ordinary Differential Equations

Consider the following ordinary differential equation (ODE):

$$\frac{dx}{dt} = - x$$

with the initial condition

$$x(0) = 1$$

Of course, we know the solution analytically:

$$x(t) = \exp(-t)$$

But how do we solve this numerically?  A simple approach is to use the definition of the derivative, but with the differential widened to a finite (but small) interval, to write the derivative as

$$\frac{dx}{dt} \approx \frac{x^{n+1} - x^n}{\Delta t}$$

(Alternatively, we can obtain this approximation by noting that the derivative is just the local slope, and assume that in a small interval the curve can be approximate by a line.)  Here we have replaced the continuous variable, $x(t)$, with the discrete series $x^n$, where here the superscript is the time index, not an exponent.  For simplicity, we will take the time step as a constant, such that $x^n = x(n \Delta t)$.  Substituting this expression into the original equation, we have

$$\frac{x^{n+1} - x^n}{\Delta t} = - x^n$$

This is called a *discrete* form of the equations, we say that the equations have been *discretized in time*.  Now we can solve this expression to find a rule that computes $x^{n+1}$ given a value for $x^n$.

$$x^{n+1} =  \left( 1 - \Delta t \right) x^n$$

Let's take a look at the numercal solution computed using this formula.  In what follows we will use Python with the Numpy library, and try to emphasize readability and conciseness of code over optimization.  There are many other ways to implement the same operation.  Refer to the Python and Numpy documentation for any forms you don't recognize.

In [ ]:
from matplotlib.pylab import *

In [ ]:
def ode_fe(x0, tmax, dt):
    t = [ 0 ]
    x = [ x0 ]
    while(t[-1]<=tmax):
        t.append( t[-1] + dt )
        x.append( ( 1 - dt ) * x[-1] )
    return t, x

In [ ]:
t_exact = linspace(0, 5)
x_exact = exp(-t_exact)

t, x = ode_fe(1, 5, 0.5)

plot(t, x, 'o-', label="Simulation")
plot(t_exact, x_exact, 'k', label="Analytic")
legend()

That looks okay, but the accuracy is not great.  The discrepency is clearer if you replot with a log scale in $y$ (try it, the matplotlib command is `semilogy`).  We can improve that by decreasing the timestep:

In [ ]:
for dt in [ 0.8, 0.4, 0.2, 0.1 ]:
    t, x = ode_fe(1, 5, dt)
    plot(t, x, 'o-', label="dt={}".format(dt))
plot(t_exact, x_exact, 'k', label="Analytic")
legend()

See how the results improve as the time step decreases.  Again, for this problem it is much more visible if you plot in log scale.  In fact, if we were to define an error norm, we would find that the error in the solution is proporational to the time step.  This property is called converegence, and the convergence rate for this solver is linear in time.  This is a very important property, but we will not discuss it further in these notes.

What happens if you make the time step larger?  In particular, note that something happens when $\Delta t > 1$, namely, the sign of the prefactor in the discrete evolution equation changes sign.

In [ ]:
t_exact = linspace(0, 10)
x_exact = exp(-t_exact)

t, x = ode_fe(1, 10, 1.5)

plot(t, x, 'o-', label="Simulation")
plot(t_exact, x_exact, 'k', label="Analytic")
legend()

See how the solution becomes oscillatory.  It still retains the decaying character of the true solution, but it approaches it in a pathological way.  If the time step is further increased, such that $\Delta t>2$, the prefactor is now greater than $-1$, and starts to amplify.

In [ ]:
t, x = ode_fe(1, 10, 2.5)

plot(t, x, 'o-', label="Simulation")
plot(t_exact, x_exact, 'k', label="Analytic")
legend()

This important behavior is related to the concept of stability.

The method we used to discretize the equations is called a *forward Euler* method, because it relied on discritizing the derivative by considering a difference forward in time.   We could also have done a *backward Euler* method, which considers the difference looking backward in time.  Backward Euler is an *implicit* method, because, as you will see, the right hand side needs to be evaluated at the new, as yet unknown, timestep.  For a linear equation such as ours, this does not introduce any computational difficulty.  Forward Euler is an *explicit* method, because the value at the new time is written explicitly in terms of the value at the old time.  Typically implicit methods have better stability properties than explicit methods.

We now introduce the alternate approximation for the derivative

$$\frac{dx}{dt} \approx \frac{x^n - x^{n-1}}{\Delta t}$$

Again, we substitute into the governing equation,

$$\frac{x^n - x^{n-1}}{\Delta t} = - x_n$$

and solve for the value at the new time, in terms of the value at the previous time,

$$x_n  =  \frac{x^{n-1}}{1 + \Delta t}$$

Since $n$ is an arbitrary index, for ease of comparision to the forward Euler scheme, we can replace $n$ with $n+1$, and write:

$$x_{n+1}  =  \frac{x^n}{1 + \Delta t}$$

Notice that, for the backward Euler, the multiplicative factor on the right hand side is always positive.  The result is that this scheme is stable for all values of $\Delta t$.

In [ ]:
def ode_be(x0, tmax, dt):
    t = [ 0 ]
    x = [ x0 ]
    while(t[-1]<=tmax):
        t.append( t[-1] + dt )
        x.append( x[-1] / ( 1 + dt ) )
    return t, x

In [ ]:
for dt in [ 3.2, 1.6, 0.8, 0.4, 0.2, 0.1 ]:
    t, x = ode_be(1, 10, dt)
    plot(t, x, 'o-', label="dt={}".format(dt))
plot(t_exact, x_exact, 'k', label="Analytic")
legend()

As you can see, although the solution with $\Delta t=3.2$ is a very poor approximation, it remains stable and retains the character of the true solution.  (What do I mean by the "character" of the true solution?  If you aren't sure, try plotting with a log scale in $y$ again, and see if it becomes clear.  Before you do, what do you expect the plot to look like?)

There is a lot more to say about methods for solving ODEs.  Modern ODE solution packages, such as the one included in SciPy, use much more sophisticated techniques than what we have outlined here.  However, those are well beyond the scope of these notes.  With what we have see, we are now ready to consider partial differential equations (PDEs).

# Partial Differential Equations

For what follows, we are going to consider the one-dimensional diffusion equation,

$$\frac{\partial u}{\partial t} - \alpha \frac{\partial^2 u}{\partial x^2} = 0$$

which you may recognize as the equation for the conduction of heat along a thin bar, where $u$ is the temperature and $\alpha$ is the conductivity.

This equation can be solved analytically using transform methods, or equivalently, by separation of variables.  We will look for a solution of the form

$$u(x, t) = X(x)T(t)$$

Substituting into the diffusion equation we have

$$XT' - \alpha X''T = 0$$

where the prime indicates differentiation by the lone variable.  We can now group terms with identical dependencies,

$$\frac{T'}{T} = \alpha \frac{X''}{X} = -C$$

Since the left hand side is a function of $t$ only, and the right hand side of $x$ only, they both must be equal to a constant, which we denote $C$.  (The minus sign is required to obtain real solutions when $\alpha$ and $C$ are real and positive.)  We can now write two separate equations for $X$ and $T$, which can be solved independently,

$$T' = - C T$$

so 

$$T(t) = \exp(- C t)$$

and,

$$X'' = - \frac{C}{\alpha} X$$

so

$$X(x) = A \sin \sqrt{\frac{C}{\alpha}} x + B \cos \sqrt{\frac{C}{\alpha}} x$$

Therefore

$$u(x, t) = \left( A \sin \sqrt{\frac{C}{\alpha}} x + B \cos \sqrt{\frac{C}{\alpha}} x \right) \exp(- C t)$$

where $A$, $B$, and $C$ are arbitrary constants which depend on the initial conditions.  (Strictly speaking, there should be an arbitrary constant in the solution for $T$, but we omitted it, as it can be absorbed into $A$ and $B$.)

Since the problem is linear, any linear combination of solutions will also be a solution, and we can use Fourier series to build up solutions for general initial conditions.  For simplicity, in the remainder of these notes, we will consider only the initial and boundary conditions,

$$\begin{aligned}
u(x, 0) & = \sin \pi x \\
u(0, t) & = u(1, t) = 0
\end{aligned}$$

in the domain $x \in [0,1]$.  In that case, the solution is

$$u(x, t) = \exp (- \alpha \pi^2 t) \sin \pi x$$

## Finite-Difference Methods

The solution methods that we looked at for ODEs are both finite-difference methods.  The extension to a multi-dimensional problem (in this case, one time and one space dimension) is relatively trivial.  These methods are called *finite-difference* because we replace the infinitiesimal differentials represented by the derivatives by finite (albeit small) differences.  We now introduce a discrete version of $u$ which is discretized in both time and space, such that we assume

$$u^n_i \approx u(i \Delta x, n \Delta t)$$

For this example, we will take both the time and space step as constant, although this does not need to be true in general.  (Formally, $u^n_i$ and $u(x,t)$ are completely different variables, despite re-using the name $u$ for both.  The latter is a function of two continuous variables, and the former can be thought of as a vector with index $i$ for each time step $n$.)

The only non-trivial issue for our diffusion equation is that we now need a difference form for the second derivative.  For this, use the fact that

$$\frac{\partial^2 u}{\partial x^2} = \frac{\partial}{\partial x} \left( \frac{\partial u}{\partial x} \right)$$

and use the same differencing procedure twice.  First we say:

$$\frac{\partial^2 u}{\partial x^2} \approx \frac{1}{\Delta x} \left( \left. \frac{\partial u}{\partial x} \right|_i - \left. \frac{\partial u}{\partial x} \right|_{i-1} \right)$$

Next, we say, for the inner derivative,

$$\left. \frac{\partial u}{\partial x} \right|_i \approx \frac{u_{i+1} - u_i}{\Delta x}$$

Combining these two we get the approximation

$$\frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1} - 2 u_i + u_{i-1}}{\Delta x^2}$$

Note that in deriving this approximation we used a backward difference for the outer derivative, and a forward difference for both inner derivatives.  What would happen if we used other combinations?

Now we can substitute difference experssions for the derivatives in the diffusion equation.  We will consider a forward Euler in time, although for real applications this is not a good choice because it has stability issues.

$$\frac{u^{n+1}_i - u^n_i}{\Delta t} - \alpha \frac{u_{i+1} - 2 u_i + u_{i-1}}{\Delta x^2} = 0$$

Now we solve for the values at the $n+1$ time level in terms of the $n$ time level:

$$u^{n+1}_i = u^n_i + \frac{\alpha \Delta t}{\Delta x^2} \left( u_{i+1} - 2 u_i + u_{i-1} \right)$$

You might want to try implementing this for yourself, as a programming exercise.  Here is my implementation.

In [ ]:
def conduction_fd(alpha, tmax, dt, npts):
    t = 0.0
    x = linspace(0, 1, npts+1)
    u = sin(pi*x)
    dx = 1/npts
    while(t<tmax):
        t += dt
        u[1:-1] += alpha*dt/dx**2*(u[:-2]-2*u[1:-1]+u[2:])
    return t, x, u

In [ ]:
tmax = 1
alpha = 0.01

x_exact = linspace(0, 1)
u_exact = exp(-pi**2*alpha*tmax)*sin(pi*x_exact)

t, x, u = conduction_fd(alpha, tmax, 0.1, 16)

plot(x, u, 'o-', label="FD")
plot(x_exact, u_exact, label="Exact")
legend()

A few notes on this code.  First, it does not save all the time levels.  It integrates until it reaches `tmax`, and then returns the solution at the final time.  Second, be careful, if `tmax` is not a multiple of `dt` the final time will not be `tmax`.  This is important if you are comparing to the exact solution.  Finally, although we will not prove it here, the stability criteria for this code is

$$\frac{\alpha \Delta t}{\Delta x^2} < \frac{1}{2} $$

You may want to play around with the code to see what happens when the stability criteria is not satisfied.  Hint: it takes a finite amount of time for the solution to go bad.

The finite-difference method has a fundamental connection to the Taylor series expansion.  We can try to recover the original PDE by replacing the values at discrete points with their Taylor series representations.  To do this, we will assume the existence of a continuous function (actually $C^\infty$), $v(x,t)$ which is equal to $u^n_i$ at each of the discrete points.  Since $v$ is continuous we can write

$$u^{n+1}_i = \sum_{l=0}^\infty \frac{\Delta t^l}{l!} \frac{\partial^l v}{\partial t^l}$$

and our finite difference approximation of the time derivative term becomes

$$\frac{u^{n+1}_i - u^n_i}{\Delta t} = \frac{\partial v}{\partial t} + \sum_{l=2}^\infty \frac{\Delta t^{l-1}}{l!} \frac{\partial^l v}{\partial t^l}$$

where we pull the first term out of the sum to show explicitly that our difference formula is exactly equal to the derivative we are looking for, plus a sum of higher-order terms in $\Delta t$.

Similarly, for the diffusion term we have (you should verify this equation for yourself)

$$\frac{u^n_{i+1} - 2 u^n_i + u^n_{i-1}}{\Delta x^2} = \frac{\partial^2 v}{\partial x^2} + 2 \sum_{\substack{l\ge4\\l\text{ even}}}^\infty \frac{\Delta x^{l-2}}{l!} \frac{\partial^l v}{\partial x^l}$$

Substituting these into the finite-difference equation we have the following equation,

$$\frac{\partial v}{\partial t} + \sum_{l=2}^\infty \frac{\Delta t^{l-1}}{l!} \frac{\partial^l v}{\partial t^l}
- \alpha \left( \frac{\partial^2 v}{\partial x^2} + 2 \sum_{\substack{l\ge4\\l\text{ even}}}^\infty \frac{\Delta x^{l-2}}{l!} \frac{\partial^l v}{\partial x^l} \right) = 0$$

which, to leading order, is

$$\frac{\partial v}{\partial t} - \alpha \frac{\partial^2 v}{\partial x^2}
= 2 \alpha \frac{\Delta x^2}{24} \frac{\partial^4 v}{\partial x^4}
- \frac{\Delta t}{2} \frac{\partial^2 v}{\partial t^2} + \text{higher order terms}$$

This is called the modifed equation.  The modified equation is a differential equation, the solution of which exactly corresponds to the solution of our finite-difference equation at the grid points.  Notice that the left hand side is exactly our original heat equation.  The right hand side is called the truncation error, and it has terms proportional to $\Delta t$ and $\Delta x^2$.  This means that the modified equation converges to the original exact equation when the time and space steps go to zero.  It also tells us that this method is first-order in space, and second order in time.

## Finite-Volume Method

Just as the finite-difference method is fundementally linked to the Taylor series, the finite-volume method is connected to the divergence theorem.  To refresh your memory, the divergence theorem says that a volume integral of a divergence is equal to the surface integral of the flux,

$$\int_V \nabla \cdot \mathbf{F} dV = \int_S \mathbf{F} \cdot d\mathbf{S}$$

where $\mathbf{F}$ is some vector function, $dV$ is a volume element, and $d\mathbf{S}$ is a differential surface normal.  As written, the theorem applies regardless of the dimensionality of the vector field.

Unlike the finite-difference method, where we considered the solution at a discrete set of point, we now divide the domain into a set of discrete volumes, which we will call *cells*.  In a higher-dimensional problem the cells may take on many possible shapes, but for our one-dimensional example, the cells will just be the regions between the uniformly distributed points $i \Delta x$.  The $i$-th cell is the region bounded by $x \in [(i-1) \Delta x, i \Delta x]$.

Let's consider our diffusion equation, and integrate over the $i$-th cell, normalizing by the cell volume (which in one-dimension is just $\Delta x$)

$$\frac{1}{\Delta x} \int_{x_{i-1}}^{x_i} \left( \frac{\partial u}{\partial t} - \alpha \frac{\partial^2 u}{\partial x^2} \right) dx = 0$$

The integral of the first term commutes with the time derivative.  For the second term, we use the divergence theorem, which in one-dimension reduces to the second fundemental theorem of calculus.  The resulting equation is

$$\frac{\partial}{\partial t} \left[ \frac{1}{\Delta x} \int_{x_{i-1}}^{x_i} u \, dx \right]
- \frac{\alpha}{\Delta x} \left. \frac{\partial u}{\partial x} \right|_{x_{i-1}}^{x_i} = 0$$

The quantity inside the time derivative is the cell averaged value of $u$ in the $i$-th cell, which we will denote $\bar{u}_i$.  In the limit of very small cells, we note that $u(x,t) \approx \bar{u}_i$ for $x$ inside the $i$-th cell.  The second term is the sum of the fluxes on the two faces.

This equation says that the time rate of change of the average temperature inside the cell is equal the flux through the sides of the cell.  In order to solve the equation, we need an approximation for the flux terms on the cell boundaries (faces).  First, let's consider the right face of the $i$-th cell, which is located at $x = i \Delta x$.  If we interpret the cell averaged values as existing at the centroid of the cells, we conviniently have the velocities available at $x = \left( i \pm \frac{1}{2} \right) \Delta t$.  The are $\bar{u}_{i+1}$ and $\bar{u}_i$.  To get the flux at the face, we can use the finite-difference form

$$\left. \frac{\partial u}{\partial x} \right|_{x=x_i}
\approx \frac{\bar{u}_{i+1} - \bar{u}_i}{\Delta x}$$

Note that if the flux was not a derivative (for example, for the wave equation the flux is $u$ not $\partial u/\partial x$), then we would need to interpolate the cell centered values to get the flux on the face.

The flux at the left face, $x = ( i - 1 ) \Delta x$ is found the same way (try it).

Substituting these fluxes into our finite-volume equation, we have

$$\frac{\partial \bar{u}_i}{\partial t}
- \frac{\alpha}{\Delta x} \left( \frac{\bar{u}_{i+1} - \bar{u}_i}{\Delta x} - \frac{\bar{u}_{i} - \bar{u}_{i-1}}{\Delta x} \right) = 0$$

or

$$\frac{\partial \bar{u}_i}{\partial t}
- \frac{\alpha}{\Delta x^2} \left( \bar{u}_{i+1} - 2 \bar{u}_i + \bar{u}_{i-1} \right) = 0$$

If we now apply a forward Euler scheme to the time derivative, this equation becomes

$$\frac{\bar{u}_i^{n+1} - \bar{u}_i^n}{\Delta t}
- \frac{\alpha}{\Delta x^2} \left( \bar{u}_{i+1}^n - 2 \bar{u}_i^n + \bar{u}_{i-1}^n \right) = 0$$

which interestingly, is the identical to the difference equation we obtained using the finite-difference method.  This is why, in the words of Professor Alfred Schatz, "the finite-difference method and the finite-volume methods are brothers."  Note that, in general, this equivalence does not hold.  That is, given a specific set approximations for the finite-difference stencils or finite-volume fluxes, there may not be a convinient way to express one in terms of the other.  The example here was chosen specifically to illustrate how the two methods are connected.  You may wish to try deriving a scheme using a different approximation to see how the methods can differ.

Another thing to note in this case is that, the way we have constructed this method, the velocity at the boundary is not represented.  That is because the velocity data is *cell centered* not *face* or *node centered*, as it was in the finite-difference method.  This will require a special boundary treatment.  For the left face of the first cell, and for the right face of the last cell, the difference stencil we have obtained for computing the flux will not work, because it requires data from a cell outside the computation.  The fluxes on the boundaries must be computed using specialized stencils which incorporate the boundary condition $u(0, t) = u(1, t) = 0$.

## Finite-Element Method

The just like the finite-difference and finite-volume methods, the finite-element method is linked to a specific mathematical formulation, in this case, the weak form of the equation.  A full treatment of the weak form of a differential equation is beyond the scope of this work, and we will omit some of the mathematical rigour necessary for a formally correct treatment, in favor of a user friendly approach.

To obtain the weak formulation of the equation, we multiply by a *test function* which we will denote $\phi(x)$ and integrate,

$$\int_0^1 \phi \left( \frac{\partial u}{\partial t} - \alpha \frac{\partial^2 u}{\partial x^2} \right) dx = 0$$

The idea is that we will require this integral equation to be satisfied for all functions in a space of test functions (which, in a rigourous treatment, would need to be defined).  The nice thing about this formulation is that it can be used to allow for solutions with certain types of discontinuities which render the original differential equations invalid, because the derivitives are not defined everywhere.  This is important, for example, in inviscid flows with shocks.  These solutions are said to be defined as *distributions*, which are a type of generalized functions.  The step function and the delta function are examples of distributions.

We do this by shifting the derivatives from $u$ to $\phi$ as follows.  Since $\phi$ is not a function of time, it commutes with the time derivative.  For the space integral, we can use integration by parts to transfer the derivative from $u$ to $\phi$,

$$\int_0^1 \frac{\partial \phi u}{\partial t} \, dx 
- \alpha \left[ \phi \frac{\partial u}{\partial x} \right]_0^1
+ \alpha \int_0^1 \frac{\partial \phi}{\partial x} \frac{\partial u}{\partial x} dx = 0$$

If we require the test functions $\phi$ to go to zero at the boundaries, then the second term vanishes.  Further, we require that the test functions are smooth enough to take whatever derivatives we need.  In that case, this equation is now well defined, even if the second derivative of $u$ does not exist, that is, if $u$ has a discontinous first derivative.

This is still a continuous equation.  To derive a discete form of the equation, we will restrict the test functions to a finite dimensional function space, that is, a function space with a finite basis.  In other words, we will choose a finite set of test functions, $\phi_l$, such that the space of test functions is the set of functions that can be expressed as

$$\phi(x) = \sum_l a_l \phi_l$$

where $a_l$ are arbitrary real coefficients.  We will also restrict $u$ to a finite dimensional function space.  For convinience, we will use the same space that we used for the test functions (this is not a requirement for finite-element methods), so that

$$u(x) = u_l \phi_l(x)$$

Finally, to make things really simple, we will choose for our $\phi_l$ the *tent functions* defined as

$$\phi_i = \begin{cases}
\frac{x - x_{i-1}}{\Delta x}, & x_{i-1} < x < x_i \\
\frac{x_{i+1} - x}{\Delta x}, & x_i < x < x_{i+1} \\
0
\end{cases}$$

where $x_i = i \Delta x$.  The first three tent functions are shown in the figure.

In [ ]:
x = arange(7)
for i in x[:3]:
    plot(x, x==i, 'o-', label=r"$\phi_{}$".format(i))
legend()

Note that $u(x)$ is a function of $x$, as are $\phi_l$, but $u_l$ is a scalar real number that multiplies the basis function $\phi_l$.  However, the specific basis we have chosen has the nice property that $u(x_i) = u_i$, which makes it easy to go between the vector representation of the function, and the spatial representation.

With these definitions, it is now not necessary to show that the weak form of the equation is satisfied for all $\phi$, rather it is enough to show that it is satisfied for each of the basis functions $\phi_l$.  This gives us a finite set of equations, one for each $l$.  (Note, $l \leq N$, where $N$ is the number of grid points, $1/N = \Delta x$.)  Writing down the $l$-th equation, and subsituting in for $u(x)$ in terms of the basis functions gives

$$\sum_i \frac{\partial u_i}{\partial t} ( \phi_l \circ \phi_i )
+ \alpha \sum_i u_i \left( \frac{\partial \phi_l}{\partial x} \circ \frac{\partial \phi_i}{\partial x} \right) = 0$$

Where we define the inner product

$$ u \circ v = \int_0^1 uv dx $$

You should try to derive this formula yourself.

The inner products in the above formula are all constants, and can be shown to have the following values:

$$\phi_i \circ \phi_j
= \begin{cases}
\frac{2 \Delta x}{3}, & i = j \\
\frac{\Delta x}{6}, & i = j+1 \text{ or } i = j-1 \\
0, & \text{elsewhere}
\end{cases} $$


$$ \frac{\partial \phi_i}{\partial x} \circ \frac{\partial \phi_j}{\partial x}
= \begin{cases}
\frac{2}{\Delta x}, & i = j \\
- \frac{1}{\Delta x}, & i = j+1 \text{ or } i = j-1 \\
0, & \text{elsewhere}
\end{cases} $$

We leave the derivation as an exercise of to the reader.  Hint: break the integrals into integrals over each interval, and note in which intervals the integrand is zero.  You will also use

$$\frac{\partial \phi_i}{\partial x} = \begin{cases}
\frac{1}{\Delta x}, & x_{i-1} < x < x_i \\
-\frac{1}{\Delta x}, & x_i < x < x_{i+1} \\
0, & \text{elsewhere}
\end{cases}$$

This gives us the difference equation

$$\frac{d}{dt} \left( \frac{\Delta x}{6} u_{l+1} + \frac{2 \Delta x}{3} u_i + \frac{\Delta x}{6} u_{l-1} \right)
+ \alpha \frac{- u_{l+1} + 2 u_l - u_{l-1}}{\Delta x} = 0$$

We still need to discritize in time.  In this case, we can see that even the forward Euler method will result in an implicit method, so we might as well use backward Euler.  The final equation is then 

$$\left( \frac{1}{6} - \frac{\alpha \Delta t}{\Delta x^2} \right) u^{n+1}_{l+1} 
+ \left( \frac{2}{3} + 2 \frac{\alpha \Delta t}{\Delta x^2} \right) u^{n+1}_l 
+ \left( \frac{1}{6} - \frac{\alpha \Delta t}{\Delta x^2} \right) u^{n+1}_{l-1}
= \frac{1}{6} u^n_{l+1} + \frac{2}{3} u^n_i + \frac{1}{6} u^n_{l-1}$$

Derive this one for yourself, too.

In [ ]:
import scipy.linalg

def conduction_fem(alpha, tmax, dt, npts):
    t = 0.0
    x = linspace(0, 1, npts+1)
    u = sin(pi*x)
    dx = 1/npts
    
    # Consult the scipy.linalg.solve_banded documentation
    # for the format of these diagonal matricies.
    diag = ones(npts+1)
    diag[1:-1] = 2/3 + 2*alpha*dt/dx**2
    supdiag = zeros(npts+1)
    supdiag[2:] = 1/6 - alpha*dt/dx**2
    subdiag = zeros(npts+1)
    subdiag[:-2] = 1/6 - alpha*dt/dx**2
    
    M = array([supdiag, diag, subdiag])
    
    while(t<tmax):
        t += dt
        u[1:-1] = 1/6*u[:-2] + 2/3*u[1:-1] + 1/6*u[2:]
        u = scipy.linalg.solve_banded((1,1), M, u)
    return t, x, u

In [ ]:
tmax = 1
alpha = 0.01

x_exact = linspace(0, 1)
u_exact = exp(-pi**2*alpha*tmax)*sin(pi*x_exact)

t, x, u = conduction_fem(alpha, tmax, 0.1, 32)

plot(x, u, 'o-', label="FEM")
plot(x_exact, u_exact, label="Exact")
legend()

A few points to note about this formulation.

First of all, the stencil for the spacial difference is again the same as for the finite-difference method.  In other words, finite-difference, finite-volume, and finite-element are all brothers.  Certain methods are easier to derive in one form rather than the other.  So there are definite advantages and disadvanteges for each, and they are not equivalent in general.

Second, if you look at the stencil inside the time-deriviative in deriving the finite-element scheme, you will see it is just a weighted average of the nearest neighbors.  If we replace that stencil with $u_i$, we recover the original finite-difference scheme exactly.  Alternatively if we conside the Laplace equation, by setting the time-derivative term to zero, the three schemes are identical.  Again, that is true in this case, we could choose other stencils, flux interpolators, or elements to get radically different schemes.

Finally, in our example we switched from an explicit method (forward Euler) to an implicit method (backward Euler).  You may want to play with the resolution to see what the stability characteristics of this scheme are.

# Conclusions

Hopefully, this gives you a unifying overview of numerical PDE solvers.  The purpose is to give you a roadmap of where to start as you learn more about this topic, or as you work with it this summer.  To that end, I conclude with some idea of what has been left out, and particularly, what would come next to get from here to a real problem (aside for just doing it all again with more rigor).

First, real problems will typically two- or three-dimensional.  This leads to more complex stencils, and the final discritized equations are generally matrix equations, like our backward Euler finite-element methods.  Unlike that method, however, higher dimensional problems will not have simple banded matricies.  They may be block diagonal, and have complex banding structure, for structured (rectangular grid).  But grids don't have to lay out in rectangular fashion where each direction on the grid corresponds to an index in an array.  Unstructured grids may have complex connectivity that leads to very general, but sparse, matricies.  Efficient techniques for solving such matricies, both direct and, more often, iterative, form a significant part of numerical methods.

Second, interesting equations are usually non-linear.  Since we have efficient methods for linear matrix solves, but not for general non-linear algebraic equations, you need to know how to convert your non-linear equation into an approximate linear form.

Third, to minimize the error without making the computation too expensive, you need to derive new schemes.  For finite-difference and finite-volume methods this usually includes going to higher-order stencils and interpolations.  For finite-element this may also include higher-order elements, or other types of elements, like spectral elements.

Fourth, in order to sufficiently resolve areas of interest in a simulation, you need to optimize grids.  This includes two areas of effort: first, designing schemes that can handle complex meshes, and second, generating those meshes.  The latter is much harder than you might think, and has become a research area of its own.  Also, for some problems it may be helpful to use a grid that moves, which introduces another whole layer of complexity.

Fifth, in today's world most large problems are run on highly parallel computers.  Designing algorithms for all of the above which scale well on today's supercomputers is yet another vital field of research.

Finally, once you have a code that does all of that, you need to make sure it is correct.  Although too often neglected, the field of numerical verification can both identify and diagnose bugs in your code, and quantify errors in the solution you compute.

In short, a great deal goes into obtaining a numerical solution to a problem.  But new computers and computational techniques open up whole new areas for our investigation.  Good luck, and enjoy!